In [ ]:
######################################CONSTANTS######################################
METRIC = 'accuracy'
MODE = 'min'

RUNS = 100
LOG_FREQ = 100
threshold = 0.98  # threshold for x-axis cutoff


COLOR = {'non-active_no_prior': '#1f77b4',
         'ts_uniform': 'red',#'#ff7f0e',
         'ts_informed': 'green',
         'epsilon_greedy_no_prior': 'tab:pink',
         'bayesian_ucb_no_prior': 'cyan',
         }


TOPK_METHOD_NAME_DICT = {'non-active_no_prior': 'Non-active',
                          #                         'non-active_uniform': 'non-active_uniform',
                          #                         'non-active_informed': 'non-active_informed',
                          'ts_uniform': 'MP-TS',
                          'ts_informed': 'MP-TS (informative)',
                          'epsilon_greedy_no_prior': 'Epsilon greedy',
                          'bayesian_ucb_no_prior': 'Bayesian UCB',
                          }

LINEWIDTH = 13.97


######################################CONSTANTS######################################
import sys

sys.path.insert(0, '..')
import argparse
from typing import Dict, Any

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

from data_utils import DATASIZE_DICT, FIGURE_DIR, RESULTS_DIR
from data_utils import DATASET_NAMES, TOPK_DICT

import matplotlib;matplotlib.rcParams['font.family'] = 'serif'

RESULTS_DIR = RESULTS_DIR + 'active_learning_topk/'



In [ ]:
def plot_topk_accuracy(ax: mpl.axes.Axes,
                       experiment_name: str,
                       topk: int,
                       eval_metric: str,
                       pool_size: int,
                       threshold: float,
                       plot_informed: bool = False) -> None:
    
    if plot_informed:
        benchmark = 'ts_informed'
        method_list = {'ts_informed': 'TS (informative)',
                       'ts_uniform': 'TS (uninformative)', }
    else:
        benchmark = 'ts_uniform'
        method_list = {#'non-active_no_prior', 
                       'ts_uniform', 
                       'epsilon_greedy_no_prior', 
                       'bayesian_ucb_no_prior'}
        # method_list = {'non-active_no_prior', 'ts_uniform'}

    for method in method_list:
        metric_eval = np.load(
            RESULTS_DIR + experiment_name + ('%s_%s.npy' % (eval_metric, method))).mean(axis=0)
        x = np.arange(len(metric_eval)) * LOG_FREQ / pool_size
        if topk == 1:
            if plot_informed:
                label = method_list[method]
            else:
                label = METHOD_NAME_DICT[method]
        else:
            label = TOPK_METHOD_NAME_DICT[method]
        ax.plot(x, metric_eval, label=label, color=COLOR[method], linewidth=3)

        if method == benchmark:
            if method == benchmark:
                if max(metric_eval) > threshold:
                    cutoff = list(map(lambda i: i > threshold, metric_eval.tolist()[10:])).index(True) + 10
                    cutoff = min(int(cutoff * 1.2), len(metric_eval) - 1)
                else:
                    cutoff = len(metric_eval) - 1

    ax.set_xlim(0, cutoff * LOG_FREQ / pool_size)
    ax.set_ylim(0, 1.0)
    xmin, xmax = ax.get_xlim()
    step = ((xmax - xmin) / 4.0001)
    ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1))
    ax.xaxis.set_ticks(np.arange(xmin, xmax + 0.001, step))
    ax.yaxis.set_ticks(np.arange(0, 1.01, 0.20))
    ax.tick_params(pad=0.25, length=1.5)

    return ax